# Merging dataframes

In the last chapter we saw how to add rows and columns to our dataframes. Merging can have the same effect of adding rows and columns, but it is a different kind of operation. When we merge dataframes, we combine rows by matching values from one column in the first dataframe to a corresponding column in the second.

In [1]:
import pandas as pd
from audiolabel import read_label
from phonlab.utils import phonemap

The first dataframe is the phone tier from a set of textgrids. The phone value is stored in the `label` column.

In [2]:
files = ('../resource/two_plus_two_1.tg', '../resource/three_plus_five_1.tg')
[phdf, wddf] = read_label(files, 'praat', addcols=['fidx'])
phdf.tail()

,t1,t2,label,fname,fidx
34,1.6986,1.7685,L,../resource/three_plus_five_1.tg,1
35,1.7685,1.8583,Z,../resource/three_plus_five_1.tg,1
36,1.8583,2.1376,EY1,../resource/three_plus_five_1.tg,1
37,2.1376,2.2274,T,../resource/three_plus_five_1.tg,1
38,2.2274,2.5966,sp,../resource/three_plus_five_1.tg,1


For future use, we'll make a copy of `phdf` and its index, and remember the number of rows. The copy is so that we can undo an intentional mistake we will make.

In [3]:
phdfcopy = phdf.copy()
origindex = phdf.index.copy()
len(phdf)

39

We'll load a second dataframe that maps arpabet phone symbols to IPA symbols. The `phonemap()` function is found in the [`phonlab` repository](https://github.com/rsprouse/phonlab).

In [4]:
ph2ipa = phonemap('arpabet', 'ipa')
ph2ipa

,arpabet,ipa
0,AW,aʊ
1,AY,aɪ
2,AA,ɑ
3,AE,æ
4,B,b
5,D,d
6,DH,ð
7,EY,eɪ
8,EH,ɛ
9,AX,ə


We'll merge `ph2ipa` into `phdf`, and we want to match rows based on `phdf`'s `label` column and `ph2ipa`'s `arpabet` column. Look at the order of the dataframes in the `merge()` statement. The first one (`phdf`) is called the `left` dataframe and the second (`ph2ipa`) is called the `right` dataframe. Use the `left_on` and `right_on` parameters to identify which column to use from each dataframe for row matching.

In [5]:
# Warning. This solution is incomplete. Read on for more.
phdf = phdf.merge(ph2ipa, left_on='label', right_on='arpabet')
phdf.tail()

,t1,t2,label,fname,fidx,arpabet,ipa
18,0.8107,0.8805,F,../resource/three_plus_five_1.tg,1,F,f
19,2.0678,2.1975,R,../resource/two_plus_two_1.tg,0,R,ɹ
20,0.1222,0.2220,R,../resource/three_plus_five_1.tg,1,R,ɹ
21,0.0125,0.1222,TH,../resource/three_plus_five_1.tg,1,TH,θ
22,1.2397,1.2696,V,../resource/three_plus_five_1.tg,1,V,v


You have to be careful when you `merge()` to ensure that you don't accidentally add or drop rows. Our intent was to keep all of our existing `phdf` rows and add the corresponding IPA symbol. Let's check how many rows are in `phdf` now.

In [6]:
len(phdf)

23

It looks like some rows were dropped since there were originally 39 rows. The original had rows with the label 'sp', and that symbol is not in `ph2ipa`. Let's do a quick check to see whether any of the rows in the new dataframe have the label 'sp'.

In [7]:
(phdf.label == 'sp').any()

False

The equality comparison returns a boolean `pd.Series`, and the `any()` method returns True if any value in the series is True or False if none are True.

The problem is that by default `merge()` returns the intersection of the matching columns, which means rows from either dataframe that don't match a row in the other dataframe are dropped. In this case we lost all of the 'sp' labels, which we might not intend to do. We can change this default behavior with the `how` parameter, which allows one of four values (default is `inner`):

* 'left': use only keys from left frame, similar to a SQL left outer join; preserve key order
* 'right': use only keys from right frame, similar to a SQL right outer join; preserve key order
* 'outer': use union of keys from both frames, similar to a SQL full outer join; sort keys lexicographically
* 'inner': use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys

For our purpose we require the `left` merge, which ensures that all of the keys from the left dataframe `phdf` are returned in the merged dataframe.

In [8]:
phdf = phdfcopy.copy()   # Reset to original phdf
phdf = phdf.merge(ph2ipa, left_on='label', right_on='arpabet', how='left')
phdf.tail()

,t1,t2,label,fname,fidx,arpabet,ipa
34,1.6986,1.7685,L,../resource/three_plus_five_1.tg,1,L,l
35,1.7685,1.8583,Z,../resource/three_plus_five_1.tg,1,Z,z
36,1.8583,2.1376,EY1,../resource/three_plus_five_1.tg,1,NaN,NaN
37,2.1376,2.2274,T,../resource/three_plus_five_1.tg,1,T,t
38,2.2274,2.5966,sp,../resource/three_plus_five_1.tg,1,NaN,NaN


The 'sp' label has returned. You can see that the non-match of 'sp' results in the null value `NaN` being filled in for the columns derived from `ph2ipa`. The number of rows is also now correct.

In [9]:
len(phdf)

39

## `merge_asof()`

The [`merge_asof()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.merge_asof.html#pandas.merge_asof) method was introduced in pandas 0.19.0, and it is very useful for phonetic datasets. A normal merge requires exact matching keys, but a lot of phonetic datasets are time-based and don't work well with `merge()`, and `merge_asof()` is a much better fit.

We already have a couple of dataframes that include aligned `word` and `phone` labels. The rows from these dataframes could be concatenated, but the result is not very useful. A better way to combine these dataframes is to create new observations (rows) that include the phone and its word context.

The `merge_asof()` method allows us to merge dataframes based on nearest key rather than equal keys. Our goal is to match each phone to the nearest word label. We do this by making `phdf` the left dataframe and using the `t1` column from both dataframes as the merge key. To keep things simple we will only look at the data from a single textgrid.

In [10]:
wddf1 = wddf.loc[wddf.fidx==1,:]
phdf1 = phdf.loc[phdf.fidx==1,:]
phwddf1 = pd.merge_asof(phdf1, wddf1, on='t1')
phwddf1.head(10)

,t1,t2_x,label_x,fname_x,fidx_x,arpabet,ipa,t2_y,label_y,fname_y,fidx_y
0,0.0125,0.1222,TH,../resource/three_plus_five_1.tg,1,TH,θ,0.4116,THREE,../resource/three_plus_five_1.tg,1
1,0.1222,0.2220,R,../resource/three_plus_five_1.tg,1,R,ɹ,0.4116,THREE,../resource/three_plus_five_1.tg,1
2,0.2220,0.4116,IY1,../resource/three_plus_five_1.tg,1,NaN,NaN,0.4116,THREE,../resource/three_plus_five_1.tg,1
3,0.4116,0.5113,P,../resource/three_plus_five_1.tg,1,P,p,0.8107,PLUS,../resource/three_plus_five_1.tg,1
4,0.5113,0.5512,L,../resource/three_plus_five_1.tg,1,L,l,0.8107,PLUS,../resource/three_plus_five_1.tg,1
5,0.5512,0.6111,AH1,../resource/three_plus_five_1.tg,1,NaN,NaN,0.8107,PLUS,../resource/three_plus_five_1.tg,1
6,0.6111,0.8107,S,../resource/three_plus_five_1.tg,1,S,s,0.8107,PLUS,../resource/three_plus_five_1.tg,1
7,0.8107,0.8805,F,../resource/three_plus_five_1.tg,1,F,f,1.2696,FIVE,../resource/three_plus_five_1.tg,1
8,0.8805,1.2397,AY1,../resource/three_plus_five_1.tg,1,NaN,NaN,1.2696,FIVE,../resource/three_plus_five_1.tg,1
9,1.2397,1.2696,V,../resource/three_plus_five_1.tg,1,V,v,1.2696,FIVE,../resource/three_plus_five_1.tg,1


The rows of the merged dataframe match each phone with the word it occurs in, based on the `t1` columns. By default `merge_asof()` matches in the 'backward' direction, defined as:

* A “backward” search selects the last row in the right DataFrame whose ‘on’ key is less than or equal to the left’s key.

For our dataset this direction matches the last word whose beginning is less than or equal to the beginning of each phone. 

Notice the column names that result from the merge. Identical column names from each input frame have suffixes in the merged dataframe that indicate which they came from. The `_x` suffix is used for the left dataframe, and `_y` is for the right. You can make the suffixes more meaningful with the suffixes parameter. The following uses `_ph` for the phone dataframe and `_wd` for the word. We'll also use column selection to remove unhelpful duplicated columns from `wddf1`.

In [11]:
wddf1 = wddf.loc[wddf.fidx==1,['t1', 't2', 'label']]
phdf1 = phdf.loc[phdf.fidx==1,:]
phwddf1 = pd.merge_asof(phdf1, wddf1, on='t1', suffixes=['_ph', '_wd'])
phwddf1.head(10)

,t1,t2_ph,label_ph,fname,fidx,arpabet,ipa,t2_wd,label_wd
0,0.0125,0.1222,TH,../resource/three_plus_five_1.tg,1,TH,θ,0.4116,THREE
1,0.1222,0.2220,R,../resource/three_plus_five_1.tg,1,R,ɹ,0.4116,THREE
2,0.2220,0.4116,IY1,../resource/three_plus_five_1.tg,1,NaN,NaN,0.4116,THREE
3,0.4116,0.5113,P,../resource/three_plus_five_1.tg,1,P,p,0.8107,PLUS
4,0.5113,0.5512,L,../resource/three_plus_five_1.tg,1,L,l,0.8107,PLUS
5,0.5512,0.6111,AH1,../resource/three_plus_five_1.tg,1,NaN,NaN,0.8107,PLUS
6,0.6111,0.8107,S,../resource/three_plus_five_1.tg,1,S,s,0.8107,PLUS
7,0.8107,0.8805,F,../resource/three_plus_five_1.tg,1,F,f,1.2696,FIVE
8,0.8805,1.2397,AY1,../resource/three_plus_five_1.tg,1,NaN,NaN,1.2696,FIVE
9,1.2397,1.2696,V,../resource/three_plus_five_1.tg,1,V,v,1.2696,FIVE


The result is a neater merged dataframe. Notice the result of merging the `t1` columns means the `t1` values from `phdf1` are retained in the rows and not `wddf1` values. If you want to preserve the word `t1` column (if you want to calculate word duration, for instance), you can rename the `t1` columns before the merge. If you do this you will also need to specify the `on` columns separately via `left_on` and `right_on`.

In [12]:
wddf1 = wddf.loc[wddf.fidx==1,['t1', 't2', 'label']]
phdf1 = phdf.loc[phdf.fidx==1,:]
phwddf1 = pd.merge_asof(
    phdf1.rename(columns={'t1': 't1_ph'}),
    wddf1.rename(columns={'t1': 't1_wd'}),
    left_on='t1_ph',
    right_on='t1_wd',
    suffixes=['_ph', '_wd']
)
phwddf1.head(10)

,t1_ph,t2_ph,label_ph,fname,fidx,arpabet,ipa,t1_wd,t2_wd,label_wd
0,0.0125,0.1222,TH,../resource/three_plus_five_1.tg,1,TH,θ,0.0125,0.4116,THREE
1,0.1222,0.2220,R,../resource/three_plus_five_1.tg,1,R,ɹ,0.0125,0.4116,THREE
2,0.2220,0.4116,IY1,../resource/three_plus_five_1.tg,1,NaN,NaN,0.0125,0.4116,THREE
3,0.4116,0.5113,P,../resource/three_plus_five_1.tg,1,P,p,0.4116,0.8107,PLUS
4,0.5113,0.5512,L,../resource/three_plus_five_1.tg,1,L,l,0.4116,0.8107,PLUS
5,0.5512,0.6111,AH1,../resource/three_plus_five_1.tg,1,NaN,NaN,0.4116,0.8107,PLUS
6,0.6111,0.8107,S,../resource/three_plus_five_1.tg,1,S,s,0.4116,0.8107,PLUS
7,0.8107,0.8805,F,../resource/three_plus_five_1.tg,1,F,f,0.8107,1.2696,FIVE
8,0.8805,1.2397,AY1,../resource/three_plus_five_1.tg,1,NaN,NaN,0.8107,1.2696,FIVE
9,1.2397,1.2696,V,../resource/three_plus_five_1.tg,1,V,v,0.8107,1.2696,FIVE


Now that we have phones combined with the word context, let's try combining these with formant measurements. First load the formant measurements into a dataframe.

In [13]:
ifcfile = '../resource/three_plus_five_1.ifc'
ifcdf = pd.read_csv(ifcfile, sep='\t')
ifcdf.head()

,sec,rms,f1,f2,f3,f4,f0
0,0.005,65.2,749.8,1750.8,2493.2,3667.3,0.0
1,0.015,132.1,660.6,1722.0,2450.1,3878.3,162.2
2,0.025,218.0,659.7,1707.1,2448.8,4003.2,571.4
3,0.035,294.0,673.3,1701.5,2810.1,3854.4,571.4
4,0.045,369.8,547.7,1736.6,2713.1,3711.2,571.4


We'll use `merge_asof()` to merge the labels with the measurements. To do this, we make the formant measurements the left dataframe and the labels the right. These dataframes use different names for the time columns, so we specify the merge keys for each separately.

In [14]:
df = pd.merge_asof(ifcdf, phwddf1, left_on='sec', right_on='t1_ph')
df.head(20)

,sec,rms,f1,f2,f3,f4,f0,t1_ph,t2_ph,label_ph,fname,fidx,arpabet,ipa,t1_wd,t2_wd,label_wd
0,0.005,65.2,749.8,1750.8,2493.2,3667.3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.015,132.1,660.6,1722.0,2450.1,3878.3,162.2,0.0125,0.1222,TH,../resource/three_plus_five_1.tg,1.0,TH,θ,0.0125,0.4116,THREE
2,0.025,218.0,659.7,1707.1,2448.8,4003.2,571.4,0.0125,0.1222,TH,../resource/three_plus_five_1.tg,1.0,TH,θ,0.0125,0.4116,THREE
3,0.035,294.0,673.3,1701.5,2810.1,3854.4,571.4,0.0125,0.1222,TH,../resource/three_plus_five_1.tg,1.0,TH,θ,0.0125,0.4116,THREE
4,0.045,369.8,547.7,1736.6,2713.1,3711.2,571.4,0.0125,0.1222,TH,../resource/three_plus_five_1.tg,1.0,TH,θ,0.0125,0.4116,THREE
5,0.055,529.0,605.7,1735.3,2621.7,3602.2,300.0,0.0125,0.1222,TH,../resource/three_plus_five_1.tg,1.0,TH,θ,0.0125,0.4116,THREE
6,0.065,725.9,589.0,1673.2,2673.1,3581.7,0.0,0.0125,0.1222,TH,../resource/three_plus_five_1.tg,1.0,TH,θ,0.0125,0.4116,THREE
7,0.075,725.9,494.0,1697.1,2855.9,3526.3,571.4,0.0125,0.1222,TH,../resource/three_plus_five_1.tg,1.0,TH,θ,0.0125,0.4116,THREE
8,0.085,733.9,447.6,1942.5,2643.0,3512.0,571.4,0.0125,0.1222,TH,../resource/three_plus_five_1.tg,1.0,TH,θ,0.0125,0.4116,THREE
9,0.095,830.2,332.2,1836.9,2434.4,3586.7,244.9,0.0125,0.1222,TH,../resource/three_plus_five_1.tg,1.0,TH,θ,0.0125,0.4116,THREE


The final merged dataframe consists of complex observations--formant (and f0 and rms) measurements and the phone and word contexts in which they occur. This is very useful set of observations on which to calculate statistics.

In [15]:
df.groupby(['ipa', 'label_wd']).f1.mean()

ipa  label_wd
f    FIVE        388.385714
k    EQUALS      454.571429
l    EQUALS      367.485714
     PLUS        509.850000
p    PLUS        627.150000
s    PLUS        529.470000
t    EIGHT       504.211111
v    FIVE        516.766667
w    EQUALS      366.266667
z    EQUALS      485.088889
ɹ    THREE       339.840000
θ    THREE       514.890909
Name: f1, dtype: float64

## `merge()` and `merge_asof()` on groups of files

Let's get a little more complicated. The previous example illustrated how to merge phone and word tiers from a single textgrid. But what if we have loaded a large number of textgrids into phone and word tier dataframes? Can we still merge them with `merge_asof()`? Consider the following example, which contains phones and words from two textgrids.

In [16]:
files = ('../resource/two_plus_two_1.tg', '../resource/three_plus_five_1.tg')
[phdf, wddf] = read_label(files, 'praat', addcols=['fidx'])

`merge_asof()` allows two different kinds of row matching. You can specify matching on equivalent keys with the `by` parameter  (or `left_by`/`right_by`) in addition to the time-based matching via the `on` parameter (or `left_on`/`right_on`). In the cell below we require **exact** matches of the `fname` column and **nearest-time** matches of the `t1_ph` and `t1_wd` columns.

Notice one other important detail: `merge_asof()` requires that the columns identified by the `on` parameter be sorted. The `t1` columns in the dataframes returned by `read_label()` are sorted first by textgrid, then by time, and we must sort by the `t1` columns first when performing `merge_asof()` to avoid raising an error.

In [17]:
phwddf = pd.merge_asof(
    phdf.rename(columns={'t1': 't1_ph'}).sort_values('t1_ph'),
    wddf.rename(columns={'t1': 't1_wd'}).sort_values('t1_wd'),
    by='fname',
    left_on='t1_ph',
    right_on='t1_wd',
    suffixes=['_ph', '_wd']
)
phwddf.head(10)

,t1_ph,t2_ph,label_ph,fname,fidx_ph,t1_wd,t2_wd,label_wd,fidx_wd
0,0.0125,0.3417,T,../resource/two_plus_two_1.tg,0,0.0125,0.4914,TWO,0
1,0.0125,0.1222,TH,../resource/three_plus_five_1.tg,1,0.0125,0.4116,THREE,1
2,0.1222,0.2220,R,../resource/three_plus_five_1.tg,1,0.0125,0.4116,THREE,1
3,0.2220,0.4116,IY1,../resource/three_plus_five_1.tg,1,0.0125,0.4116,THREE,1
4,0.3417,0.4914,UW1,../resource/two_plus_two_1.tg,0,0.0125,0.4914,TWO,0
5,0.4116,0.5113,P,../resource/three_plus_five_1.tg,1,0.4116,0.8107,PLUS,1
6,0.4914,0.5912,P,../resource/two_plus_two_1.tg,0,0.4914,0.8805,PLUS,0
7,0.5113,0.5512,L,../resource/three_plus_five_1.tg,1,0.4116,0.8107,PLUS,1
8,0.5512,0.6111,AH1,../resource/three_plus_five_1.tg,1,0.4116,0.8107,PLUS,1
9,0.5912,0.6211,L,../resource/two_plus_two_1.tg,0,0.4914,0.8805,PLUS,0


The dataframe returned by `merge_asof()` is also sorted by time, which has the effect of intermingling rows from separate textgrids. If you wish you can view them sorted first by textgrid, then by time.

In [18]:
phwddf.sort_values(['fname', 't1_ph'])

,t1_ph,t2_ph,label_ph,fname,fidx_ph,t1_wd,t2_wd,label_wd,fidx_wd
1,0.0125,0.1222,TH,../resource/three_plus_five_1.tg,1,0.0125,0.4116,THREE,1
2,0.1222,0.2220,R,../resource/three_plus_five_1.tg,1,0.0125,0.4116,THREE,1
3,0.2220,0.4116,IY1,../resource/three_plus_five_1.tg,1,0.0125,0.4116,THREE,1
5,0.4116,0.5113,P,../resource/three_plus_five_1.tg,1,0.4116,0.8107,PLUS,1
7,0.5113,0.5512,L,../resource/three_plus_five_1.tg,1,0.4116,0.8107,PLUS,1
8,0.5512,0.6111,AH1,../resource/three_plus_five_1.tg,1,0.4116,0.8107,PLUS,1
10,0.6111,0.8107,S,../resource/three_plus_five_1.tg,1,0.4116,0.8107,PLUS,1
13,0.8107,0.8805,F,../resource/three_plus_five_1.tg,1,0.8107,1.2696,FIVE,1
14,0.8805,1.2397,AY1,../resource/three_plus_five_1.tg,1,0.8107,1.2696,FIVE,1
17,1.2397,1.2696,V,../resource/three_plus_five_1.tg,1,0.8107,1.2696,FIVE,1
